In [1]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:
meta = pd.read_csv(
    Path.cwd() / "data" / "meta.csv",
    parse_dates=["first_include"],
    date_format="%Y-%m-%d",
)

In [3]:
prices = (
    pd.read_csv(Path.cwd() / "data" / "historical_prices_monthly_stat.csv")
    .sort_values(["_code", "_year", "_month"], ascending=True)
    .reset_index(drop=True)
)

mvs = (
    pd.read_csv(Path.cwd() / "data" / "historical_mvs_monthly_stat.csv")
    .sort_values(["_code", "_year", "_month"], ascending=True)
    .reset_index(drop=True)
)

In [4]:
historical = pd.merge(prices, mvs, on=['_code', '_year', '_month'], how='inner')

In [5]:
df = pd.merge(historical, meta, how="inner", on="_code")
df["ym"] = pd.to_datetime(
    df["_year"].astype(str) + df["_month"].astype(str).str.rjust(2, "0"), 
    format="%Y%m"
)
# Only use historical price data to remove survival effect
df = df[df["ym"] >= df["first_include"]].reset_index(drop=True)

In [6]:
df['monthly_rtn_davg'] = df['monthly_rtn'] / df['monthly_nbdays']
df['monthly_high_low_rtn_davg'] = df['monthly_high_low_rtn'] / df['monthly_high_low_nbdays']
df['monthly_high_end_rtn_davg'] = df['monthly_high_end_rtn'] / df['monthly_high_end_nbdays']
df['monthly_start_high_rtn_davg'] = df['monthly_start_high_rtn'] / df['monthly_start_high_nbdays']

In [7]:
df['monthly_high_position'] = df['monthly_start_high_nbdays']/df['monthly_nbdays']
df['monthly_sharpe'] = df['monthly_rtn_davg']/df['monthly_vola'].clip(lower=1e-5)
df['monthly_sortino'] = df['monthly_rtn_davg']/df['monthly_dvola'].clip(lower=1e-5)
df['monthly_sterling'] = df['monthly_rtn']/df['monthly_mdd'].clip(upper=-1e-5).abs()

In [8]:
temp = df.groupby(['_year', '_month', 'country'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].mean()
temp.columns = ['_year', '_month', 'country', 'monthly_rtn_csmean', 'monthly_start_high_rtn_csmean', 'monthly_high_end_rtn_csmean']
df = pd.merge(df, temp, on=['_year', '_month', 'country'], how='left')

temp = df.groupby(['_year', '_month', 'country'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].std()
temp.columns = ['_year', '_month', 'country', 'monthly_rtn_csstd', 'monthly_start_high_rtn_csstd', 'monthly_high_end_rtn_csstd']
df = pd.merge(df, temp, on=['_year', '_month', 'country'], how='left')

In [9]:
t3m = df.groupby(['_code'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].rolling(3, min_periods=3).mean()[['_code', 'monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']]
t3m.columns = ['_code', 'monthly_rtn_3mb_ma', 'monthly_start_high_rtn_3mb_ma', 'monthly_high_end_rtn_3mb_ma']
t3s = df.groupby(['_code'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].rolling(3, min_periods=3).std()[['_code', 'monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']]
t3s.columns = ['_code', 'monthly_rtn_3mb_ms', 'monthly_start_high_rtn_3mb_ms', 'monthly_high_end_rtn_3mb_ms']
t6m = df.groupby(['_code'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].rolling(6, min_periods=3).mean()[['_code', 'monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']]
t6m.columns = ['_code', 'monthly_rtn_6mb_ma', 'monthly_start_high_rtn_6mb_ma', 'monthly_high_end_rtn_6mb_ma']
t6s = df.groupby(['_code'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].rolling(6, min_periods=3).std()[['_code', 'monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']]
t6s.columns = ['_code', 'monthly_rtn_6mb_ms', 'monthly_start_high_rtn_6mb_ms', 'monthly_high_end_rtn_6mb_ms']
t12m = df.groupby(['_code'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].rolling(12, min_periods=3).mean()[['_code', 'monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']]
t12m.columns = ['_code', 'monthly_rtn_12mb_ma', 'monthly_start_high_rtn_12mb_ma', 'monthly_high_end_rtn_12mb_ma']
t12s = df.groupby(['_code'], as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].rolling(12, min_periods=3).std()[['_code', 'monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']]
t12s.columns = ['_code', 'monthly_rtn_12mb_ms', 'monthly_start_high_rtn_12mb_ms', 'monthly_high_end_rtn_12mb_ms']

In [10]:
df = pd.concat(
    [
        df, 
        df.groupby("_code", as_index=False)[['monthly_rtn', 'monthly_start_high_rtn', 'monthly_high_end_rtn']].shift(1).rename(columns={c: c+"_1mb" for c in df.columns}),
        t3m.groupby("_code", as_index=False).shift(1),
        t3s.groupby("_code", as_index=False).shift(1),
        t6m.groupby("_code", as_index=False).shift(1),
        t6s.groupby("_code", as_index=False).shift(1),
        t12m.groupby("_code", as_index=False).shift(1),
        t12s.groupby("_code", as_index=False).shift(1),
    ],
    axis=1,
).reset_index(drop=True)

In [11]:
VAR_PRE = [
    '_code',
    '_year',
    '_month',
    'country',
    'gics_sector',
    'monthly_market_value_avg',
    'monthly_rtn',
    'monthly_rtn_davg',
    'monthly_start_high_rtn',
    'monthly_start_high_rtn_davg',
    'monthly_high_end_rtn',
    'monthly_high_end_rtn_davg',
    'monthly_mdd',
    'monthly_vola',
    'monthly_dvola',
    'monthly_high_position', 
    'monthly_sharpe', 
    'monthly_sortino',
    'monthly_sterling', 
    'monthly_rtn_csmean',
    'monthly_start_high_rtn_csmean', 
    'monthly_high_end_rtn_csmean',
    'monthly_rtn_csstd', 
    'monthly_start_high_rtn_csstd',
    'monthly_high_end_rtn_csstd', 
    'monthly_rtn_1mb',
    'monthly_start_high_rtn_1mb', 
    'monthly_high_end_rtn_1mb',
    'monthly_rtn_3mb_ma', 
    'monthly_start_high_rtn_3mb_ma',
    'monthly_high_end_rtn_3mb_ma', 
    'monthly_rtn_3mb_ms',
    'monthly_start_high_rtn_3mb_ms', 
    'monthly_high_end_rtn_3mb_ms',
    'monthly_rtn_6mb_ma', 
    'monthly_start_high_rtn_6mb_ma',
    'monthly_high_end_rtn_6mb_ma', 
    'monthly_rtn_6mb_ms',
    'monthly_start_high_rtn_6mb_ms', 
    'monthly_high_end_rtn_6mb_ms',
    'monthly_rtn_12mb_ma', 
    'monthly_start_high_rtn_12mb_ma',
    'monthly_high_end_rtn_12mb_ma', 
    'monthly_rtn_12mb_ms',
    'monthly_start_high_rtn_12mb_ms', 
    'monthly_high_end_rtn_12mb_ms',
]

In [12]:
df = df[VAR_PRE]
df.dropna(inplace=True)

In [13]:
df = pd.concat(
    [
        df, 
        df.groupby("_code", as_index=False)[['monthly_rtn', 'monthly_start_high_rtn']].shift(-1).rename(columns={c: c + "_1mf" for c in df.columns}),
    ],
    axis=1,
).reset_index(drop=True)

In [15]:
df.dropna(inplace=True)

In [18]:
df.to_csv(Path.cwd() / "data" / "final.csv",)